<a href="https://colab.research.google.com/github/RayMess/CryptoDB/blob/main/CryptoProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [240]:
!pip install tink
!pip install passlib
!pip install argon2_cffi
!pip install bcrypt

In [241]:
from tink import daead, cleartext_keyset_handle
import passlib
import argon2
from argon2 import PasswordHasher
import bcrypt
import tink
from passlib.context import CryptContext
import binascii

In [242]:
daead.register()
keyset_handle = tink.new_keyset_handle(daead.deterministic_aead_key_templates.AES256_SIV)
with open('keyset.json', 'wt') as f:
  cleartext_keyset_handle.write(
    tink.JsonKeysetWriter(f), keyset_handle)

In [243]:
def read_keyset(filename):
  with open(filename, 'rt') as f:
    return cleartext_keyset_handle.read(
      tink.JsonKeysetReader(f.read())
    )

In [244]:
database = 'database.txt'
secret_key = read_keyset('keyset.json') # use Tink to generate your secret key here
associated_data=b'data'
daead_primitive = secret_key.primitive(daead.DeterministicAead)

In [245]:
def hash_password(pwd):
  # create CryptContext object
  context = CryptContext(
        schemes=["argon2"],
        default="argon2",
        argon2__default_rounds=1000)
  # hash password
  hash=context.hash(pwd)
  return hash


In [246]:
def encryption_machine(msg):
  # encrypt using AES-SIV
  msgb=bytes(msg,'utf-8')
  ciphertext = daead_primitive.encrypt_deterministically(msgb, associated_data)
  return ciphertext

def decryption_machine(msg):
  msgb=daead_primitive.decrypt_deterministically(msg,associated_data)
  return msgb.decode("utf-8")

In [247]:
def save_to_database(user, pwd):
  pwd1=hash_password(pwd)
  pwd2=encryption_machine(pwd1)
  pwd3=binascii.hexlify(pwd2)
  pwd4=pwd3.decode('utf-8')
  with open(database,"a") as db:
    db.write(f'{user} , {pwd4}\n')

In [248]:
def check_password(user, pwd):
  countu=1
  countp=1
  mdp=False
  # read from database
  count=0
  with open(database,"r") as db:
    for line in db.readlines():
      #and check for authentication
      if user in line :
        userdb, md = line.split(' , ')
        md1=md.encode('utf-8')
        md1=md1.strip()
        md2=binascii.unhexlify(md1)
        dpwd = decryption_machine(md2)
        return PasswordHasher().verify(dpwd,pwd)
    return mdp

In [249]:
def create_user():
    username = input("Entrez votre nom d'utilisateur: ")
    password = input("Entrez votre mot de passe: ")
    save_to_database(username, password)
    print("Utilisateur créé")

def login():
    username = input("Entrez votre nom d'utilisateur: ")
    password = input("Entrez votre mot de passe: ")
    if check_password(username, password):
        print("Connexion réussie")
    else:
        print("Veuillez réessayer")

In [250]:
def main():
    while True:
      print("Selectionnez une option:")
      print("1:Nouvel utilisateur 2:Se connecter 3:Sortie")
      option = input("Enter option: ")
      if option == "1":
          create_user()
      elif option == "2":
          login()
      elif option == "3":
          break
      else:
          print("Choisissez 1,2 ou 3")

main()

Selectionnez une option:
1:Nouvel utilisateur 2:Se connecter 3:Sortie
Enter option: 1
Entrez votre nom d'utilisateur: abc
Entrez votre mot de passe: abc
Utilisateur créé
Selectionnez une option:
1:Nouvel utilisateur 2:Se connecter 3:Sortie
Enter option: 2
Entrez votre nom d'utilisateur: abc
Entrez votre mot de passe: abc
Connexion réussie
Selectionnez une option:
1:Nouvel utilisateur 2:Se connecter 3:Sortie
Enter option: 3
